# Social Media Analytics
## Project 3
## Graph Analysis - Creating Graphs to Analyze in Gephi (Reddit Posts)
Felix Funes 20220306 | Paula Catalan 20221048 | Efstathia Styliagkatzi 20220078 | Alisson Tapia 20221156 | S M Abrar Hossain Asif 20220223

NOTE: To run properly, this code requires the latest version of the packages. If it gives an error, run "pip install --upgrade pip" and then "pip install --upgrade networkx scipy pandas-profiling numpy" in your terminal.

### Initial Setup

In [1]:
import pandas as pd
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse import coo_matrix
import numpy as np
# Import packages
import csv
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import word_tokenize
import re
from bs4 import BeautifulSoup
from rake_nltk import Rake
from nltk.corpus import stopwords

In [2]:
# Import data
df = pd.DataFrame(pd.read_excel("https://github.com/felix-funes/social-media-analytics/blob/main/iphone-subreddit-comments.xlsx?raw=true"))
df["body"] = df["body"].astype(str)

### Data Preprocessing

In [3]:
# Defining the text preprocessing function
def textPreProcess(rawText, removeHTML=True, charsToRemove = r'\?|\.|\!|\;|\.|\"|\,|\(|\)|\&|\:|\-|\$', removeNumbers=True, removeLineBreaks=False, specialCharsToRemove = r'[^\x00-\xfd]', convertToLower=True, removeConsecutiveSpaces=True):
    if type(rawText) != str:
        return rawText
    procText = rawText
        
    # Remove HTML
    if removeHTML:
        procText = BeautifulSoup(procText,'html.parser').get_text()

    # Remove punctuation and other special characters
    if len(charsToRemove)>0:
        procText = re.sub(charsToRemove,' ',procText)

    # Remove numbers
    if removeNumbers:
        procText = re.sub(r'\d+',' ',procText)

    # Remove line breaks
    if removeLineBreaks:
        procText = procText.replace('\n',' ').replace('\r', '')

    # Remove special characters
    if len(specialCharsToRemove)>0:
        procText = re.sub(specialCharsToRemove,' ',procText)

    # Normalize to lower case
    if convertToLower:
        procText = procText.lower() 

    # Replace multiple consecutive spaces with just one space
    if removeConsecutiveSpaces:
        procText = re.sub(' +', ' ', procText)

    return procText

In [4]:
# Tokenize words
def tokenize_words(words):
    if (type(words) != str) or (word_tokenize(words) == ''):
        return np.nan
    else:
        return word_tokenize(words)

In [5]:
# Create a dataframe with only the description and apply the text preprocessing function
processedReviews = pd.DataFrame(data=df.body.apply(textPreProcess, removeNumbers=False).values, index=df.index, columns=['PreProcessedText'])

c:\Users\Felix\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
c:\Users\Felix\anaconda3\lib\site-packages\bs4\__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


### Graph Creation

In [6]:
# Extracting the 'PreProcessedText' column from the 'processedReviews' DataFrame and converting it to a list
reviews = processedReviews['PreProcessedText'].tolist()

In [7]:
# Creating an empty list to store words
words = []
# Iterating over each review in the 'reviews' list
for review in reviews:
    # Converting the review text to lowercase and splitting it into individual words
    words += review.lower().split()
# Counting the frequency of each word
word_freq = Counter(words)

In [8]:
# Creating a graph
G = nx.Graph()
# Defining stop words
stop_words = set(stopwords.words('english'))
# Adding to the graph the words not in the stop words list
for word, freq in word_freq.items():
    if word not in stop_words:
        G.add_node(word, size=freq)
    else:
        continue

In [9]:
# Iterate over each review
for review in reviews:
    # Convert the review to lowercase and split it into individual words
    review_words = review.lower().split()

    # Iterate over each word in the review
    for i in range(len(review_words)):
        # Iterate over the remaining words in the review
        for j in range(i+1, len(review_words)):
            # Check if there is an edge between the two words in the graph G
            if G.has_edge(review_words[i], review_words[j]):
                # If an edge exists, increment the weight attribute of the edge by 1
                G[review_words[i]][review_words[j]]['weight'] += 1
            else:
                # If an edge does not exist, add a new edge with weight 1 to the graph G
                G.add_edge(review_words[i], review_words[j], weight=1)

In [10]:
# Create a dictionary that maps each node name to a unique integer index
node_index = {node_name: i for i, node_name in enumerate(G.nodes())}

In [11]:
# Convert the graph to a sparse matrix
n = len(G.nodes())
row, col, data = [], [], []
for u, v, d in G.edges(data=True):
    row.append(node_index[u])
    col.append(node_index[v])
    data.append(d['weight'])
mat = sp.coo_matrix((data, (row, col)), shape=(n, n)).tocsr()

In [12]:
# Calculate the pagerank
pr = nx.pagerank(G, weight='weight')

# Add the pagerank to the node attributes
for i, (node_id, _) in enumerate(G.nodes(data=True)):
    G.nodes[node_id]['pagerank'] = pr.get(i, 0)

In [13]:
# Save the data so it can be loaded into Gephi
nx.write_gexf(G, "reddit graph.gexf")